In [1]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ["OMP_NUM_THREADS"] = "4"
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer

Using TensorFlow backend.


In [2]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [1]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
# embedding_path = "../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"
# embedding_path = "../input/crawl-300d-2M.vec"
embedding_path = "../input/glove.840B.300d.txt"
embed_size = 300
max_features = 100000
max_len = 150

NameError: name 'pd' is not defined

In [4]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
train["comment_text"].fillna("no comment")
test["comment_text"].fillna("no comment")
X_train, X_valid, Y_train, Y_valid = train_test_split(train, y, test_size = 0.1)

In [5]:
raw_text_train = X_train["comment_text"].str.lower()
raw_text_valid = X_valid["comment_text"].str.lower()
raw_text_test = test["comment_text"].str.lower()

tk = Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid)
test["comment_seq"] = tk.texts_to_sequences(raw_text_test)

X_train = pad_sequences(X_train.comment_seq, maxlen = max_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen = max_len)
test = pad_sequences(test.comment_seq, maxlen = max_len)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

In [7]:
word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [11]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D
import keras.callbacks

log_filepath = "./logs/"
tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1, write_graph=True, write_images=True)
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop,tb_cb])
    model = load_model(file_path)
    return model

In [12]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)
pred = model.predict(test, batch_size = 1024, verbose = 1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143488/143613 [============================>.] - ETA: 0s - loss: 0.0561 - acc: 0.9806
 ROC-AUC - epoch: 1 - score: 0.988893
Epoch 00001: val_loss improved from inf to 0.04301, saving model to best_model.hdf5
143613/143613 [==============================] - 2075s 14ms/step - loss: 0.0561 - acc: 0.9806 - val_loss: 0.0430 - val_acc: 0.9834
Epoch 2/4
143488/143613 [============================>.] - ETA: 0s - loss: 0.0431 - acc: 0.9835
 ROC-AUC - epoch: 2 - score: 0.990296
Epoch 00002: val_loss improved from 0.04301 to 0.03998, saving model to best_model.hdf5
143613/143613 [==============================] - 2031s 14ms/step - loss: 0.0432 - acc: 0.9835 - val_loss: 0.0400 - val_acc: 0.9841
Epoch 3/4
143488/143613 [============================>.] - ETA: 0s - loss: 0.0403 - acc: 0.9844
 ROC-AUC - epoch: 3 - score: 0.990648
Epoch 00003: val_loss did not improve
143613/143613 [==============================] - 2014s 14ms/step - loss: 0.

### 1

from start code(emmbedding is not here )
https://www.kaggle.com/konohayui/bi-gru-cnn-poolings
loss: 0.0381 - acc: 0.9851 - val_loss: 0.0398 - val_acc: 0.9847

### 2

- embedding change(same kernel embedding)
- add emoji preproceccing

In [13]:
submission = pd.read_csv("../input/sample_submission.csv")
submission[list_classes] = (pred)
submission.to_csv("submission-1.csv", index = False)
print("[{}] Completed!".format(time.time() - start_time))

[8687.315685749054] Completed!


In [14]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995084,0.321109,0.959727,0.010910,0.922985,0.640437
1,0000247867823ef7,0.000130,0.000018,0.000084,0.000005,0.000112,0.000011
2,00013b17ad220c46,0.000207,0.000112,0.000252,0.000023,0.000416,0.000103
3,00017563c3f7919a,0.000047,0.000011,0.000059,0.000014,0.000062,0.000005
4,00017695ad8997eb,0.003713,0.000127,0.001249,0.000124,0.000570,0.000053
